In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
0,0,Ahipara,NZ,-35.17,173.17,52.84,84,0,7.07,clear sky,0.0,0
1,1,Rikitea,PF,-23.12,-134.97,70.41,69,100,13.60,overcast clouds,0.0,0
2,2,Hofn,IS,64.25,-15.21,53.60,81,75,2.24,broken clouds,0.0,0
3,3,Bredasdorp,ZA,-34.53,20.04,53.60,62,0,9.17,clear sky,0.0,0
4,4,Kaitangata,NZ,-46.28,169.85,42.01,74,72,11.01,broken clouds,0.0,0


In [35]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
ask_rain = input("Do you want it to be raining? (yes/no) ")
ask_snow = input("Do you want it to be snowing? (yes/no) ")

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90
Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no) no


In [36]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                       (city_data_df["Max Temp"] >= min_temp) &
                                       (city_data_df["Rain (inches)"] == 0 if ask_rain == "no" else city_data_df["Rain (inches)"] > 0) &
                                       (city_data_df["Snow (inches)"] == 0 if ask_snow == "no" else city_data_df["Snow (inches)"] > 0)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
12,12,Sorong,ID,-0.88,131.25,77.95,78,100,3.36,overcast clouds,0.0,0
15,15,Am Timan,TD,11.03,20.28,83.16,64,76,6.51,broken clouds,0.0,0
18,18,Saint George,US,37.10,-113.58,87.80,20,1,4.00,clear sky,0.0,0
25,25,Bambous Virieux,MU,-20.34,57.76,77.00,78,75,16.11,shower rain,0.0,0
28,28,Maragheh,IR,37.39,46.24,80.60,39,20,18.34,few clouds,0.0,0
29,29,Kapaa,US,22.08,-159.32,78.80,78,40,19.46,scattered clouds,0.0,0
39,39,Yurgamysh,RU,55.37,64.47,77.05,45,99,10.54,overcast clouds,0.0,0
44,44,Carnarvon,AU,-24.87,113.63,78.80,27,0,10.29,clear sky,0.0,0
47,47,Butaritari,KI,3.07,172.79,83.79,78,49,15.46,scattered clouds,0.0,0
52,52,Bari,IT,41.12,16.85,86.00,42,0,10.29,clear sky,0.0,0


In [37]:
# Counting cities in preferred_cities_df
preferred_cities_df.count()

City_ID                162
City                   162
Country                162
Lat                    162
Lng                    162
Max Temp               162
Humidity               162
Cloudiness             162
Wind Speed             162
Current Description    162
Rain (inches)          162
Snow (inches)          162
dtype: int64

In [38]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp","Current Description", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
12,Sorong,ID,77.95,overcast clouds,-0.88,131.25,
15,Am Timan,TD,83.16,broken clouds,11.03,20.28,
18,Saint George,US,87.80,clear sky,37.10,-113.58,
25,Bambous Virieux,MU,77.00,shower rain,-20.34,57.76,
28,Maragheh,IR,80.60,few clouds,37.39,46.24,
29,Kapaa,US,78.80,scattered clouds,22.08,-159.32,
39,Yurgamysh,RU,77.05,overcast clouds,55.37,64.47,
44,Carnarvon,AU,78.80,clear sky,-24.87,113.63,
47,Butaritari,KI,83.79,scattered clouds,3.07,172.79,
52,Bari,IT,86.00,clear sky,41.12,16.85,


In [39]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key}
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
# Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [41]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
12,Sorong,ID,77.95,overcast clouds,-0.88,131.25,The Waigo Hotel
15,Am Timan,TD,83.16,broken clouds,11.03,20.28,
18,Saint George,US,87.80,clear sky,37.10,-113.58,Best Western Plus Abbey Inn
25,Bambous Virieux,MU,77.00,shower rain,-20.34,57.76,Casa Tia Villa
28,Maragheh,IR,80.60,few clouds,37.39,46.24,Maragheh Grand Hotel
29,Kapaa,US,78.80,scattered clouds,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
39,Yurgamysh,RU,77.05,overcast clouds,55.37,64.47,Gostinitsa Uyut
44,Carnarvon,AU,78.80,clear sky,-24.87,113.63,Hospitality Carnarvon
47,Butaritari,KI,83.79,scattered clouds,3.07,172.79,Isles Sunset Lodge
52,Bari,IT,86.00,clear sky,41.12,16.85,Mercure Villa Romanazzi Carducci Bari


In [42]:
# Create the output file (CSV).
output_data_file = "WeatherPy_vacation.csv"
# Export the City_Data into a CSV.
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [43]:
hotel_df.count()

City                   162
Country                162
Max Temp               162
Current Description    162
Lat                    162
Lng                    162
Hotel Name             162
dtype: int64

In [44]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [45]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [53]:
# Add a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(35.0, 30.0), zoom_level=1.90)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))